In [1]:
from utils import build_url, save_json, save_execution_state, load_execution_state

API_KEY_FILE = 'api_key.txt'
REGION = 'euw1'
BASE_URL = 'api.riotgames.com'

QUEUE_TYPE = 'RANKED_SOLO_5x5'

# Ligas de las que extraer jugadores
PIPELINE = [
    {
        'url': '/lol/league/v4/challengerleagues/by-queue/{queue}',
        'name': 'CHALLENGER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/grandmasterleagues/by-queue/{queue}',
        'name': 'GRANDMASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
        
    },
    {
        'url': '/lol/league/v4/masterleagues/by-queue/{queue}',
        'name': 'MASTER',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
        }
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'I'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'II',
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'III'
        },
        'page': 1
    },
    {
        'url': '/lol/league/v4/entries/{queue}/{tier}/{division}',
        'completed': False,
        'params': {
            'queue': QUEUE_TYPE,
            'tier': 'DIAMOND',
            'division': 'IV'
        },
        'page': 1
    }
]

with open(API_KEY_FILE) as f:
    api_key = f.read()

In [2]:
import requests, copy, time
import os, shutil
import dill as pickle

start = time.time()

execution_cache_dir = 'current_execution'
leagues_state_file = f'{execution_cache_dir}/remaining_leagues.pkl'
players_file = f'{execution_cache_dir}/players.pkl'

headers = {
  'X-Riot-Token': api_key
}

os.makedirs(execution_cache_dir, exist_ok=True)
MIN_DELAY = 20/100 # esperar 20ms entre peticiones
delay = MIN_DELAY

# Leer un fichero del estado de la ejecución si existe o copiar PIPELINE
# para empezar desde 0
leagues_current_execution = load_execution_state(leagues_state_file, copy.deepcopy(PIPELINE[0:3]))
players = load_execution_state(players_file, [])

# Filtrar las fases que ya estén completas
leagues_current_execution = list(filter(lambda league: not league['completed'], leagues_current_execution))

for league in leagues_current_execution:
    # Para las ligas paginadas, pedir en bucle hasta que no se devuelvan jugadores
    # Si alguna petición falla, aumentar delay
    # Si va bien, disminuir el delay hasta el inicial
    # Al terninar cada fase o página, guardar el estado
    if 'page' in league:
        params = league['params']
        while True:
            url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
            response = requests.get(url, headers=headers, params={'page': league['page']})
            if response.status_code == 200:
                players_temp = response.json()
                if not players_temp:
                    break
                players.extend(players_temp)
                print(f"{params['tier']} {params['division']} ({league['page']}): len: {len(players_temp)} / total: {len(players)}")
                delay = max(delay/2, MIN_DELAY)
                league['page'] += 1
            else:
                retry_after = int(response.headers['Retry-After'])
                delay += retry_after
                print(f"{params['tier']} {params['division']} ({league['page']}): Petición fallida: {response.text}")
                print(f'Esperando {retry_after}s')
            save_execution_state(leagues_current_execution, leagues_state_file)
            save_execution_state(players_file, players)
            time.sleep(delay)
    else:
        url = build_url(REGION, BASE_URL, league['url'].format(**league['params']))
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            players_temp = response.json()['entries']
            players.extend(players_temp)
            print(f"{league['name']}: len: {len(players_temp)} / total: {len(players)}")
            delay = max(delay/2, MIN_DELAY)
        else:
            retry_after = int(response.headers['Retry-After'])
            delay += retry_after
            print(f"{params['name']}: Petición fallida: {response.text}")
            print(f'Esperando {retry_after}s')
        time.sleep(delay)
    league['completed'] = True
    save_execution_state(leagues_current_execution, leagues_state_file)
    save_execution_state(players, players_file)
        
end = time.time()
print(f'\nEl proceso ha tardado {end - start}s')

CHALLENGER: len: 300 / total: 300
GRANDMASTER: len: 700 / total: 1000
MASTER: len: 5168 / total: 6168

El proceso ha tardado 2.8560590744018555s


In [3]:
from utils import call_with_retry

# Guardar añadir el accountId para poder pedir partidas
summoners = {player['summonerId'] for player in players[0:100]} # Para pruebas limitar a 100 jugadores

accounts_file = f'{execution_cache_dir}/accounts.pkl'
summoners_with_account_file = f'{execution_cache_dir}/summoners_with_account.pkl'
  
accounts = load_execution_state(accounts_file, [])
summoners_with_account = load_execution_state(summoners_with_account_file, set())

url = '/lol/summoner/v4/summoners/{}'

summoners = summoners - summoners_with_account

for summoner in summoners:
    full_url = build_url(REGION, BASE_URL, url).format(summoner)
    account = call_with_retry(full_url, headers)
    print(account['accountId'])
    accounts.append({
        'summoner': summoner,
        'account': account['accountId']
    })
    summoners_with_account.add(summoner)
    save_execution_state(accounts, accounts_file)
    save_execution_state(summoners_with_account, summoners_with_account_file)
    time.sleep(delay)

save_json(accounts, 'accounts.json')

cpDa6rnHFIQP33_P-Thda6DBEFZQQCFYNIbcnClYbXKGrEc
ypbT3klYhns-5YIPHHeZMm_YSLq06hGEqC1DU9qB_oFPkzI
ciMNptPkXwuQcRPekdtZgf8BSZ4hRPuN_lmeNPGfSr9vSLg
XpcM677yGPo2MQvoDUzBxjmE6JwzUlU-qtHDfikrWBmxL5Q
C7Ul2PgEW9by___3zSy7VFe-lN10Q8r-fiTe2kx4muvD-DTc-MHVn7kY
mw0YEQVwXJDuQ_F7WiyLqssFON_S5H0k7lfSXxbzRJg_4ARg1MhfmWKn
a_DtSjw7ZpQgGknDlTiv1Kw_mUxp9Mezl3vsnAu_QFEyLvQ
gSfZtRohb2uZvXVOCxDugSQJy9_3tsoHEJ32ZTalphFisCM
39m7Ni7N_dCGaIz0GVCTYFWZwqKzXDIY_7PyRe1jIp88TQL-tcRLz3fJ
cO8JxYUdQKPCh3XQczY3Hql71bVpr0kyH2C7eGyvcZUPdFw
Pfqyq62R18zEpnOWOOGGzdBZJROTadRD3PxIPXijbR5JB5cTfS2Ha62R
0OFixXhVOVbVW05rtl9vcgNzS11bBMdqyv8Ue8wqxp_W0vM
sgOsfWWrzNxvf5fT1PpADmcAoGCWseybtkqm1uNte1wavtN76C3bN9Wi
2kY-7xNTkviLfjtHmNdqb6TsRyMKPTEKQTWOYUTdILVWxZ--APYUiDs1
KaVQAgW7ZBJ-17xrB8owq4fkyvNuDmqn-mZRX9x-cOeGWLI
cvm__JIW5qkGlmfACqS5mJo5K6fMTFdEFwExal23cr0_W50
PIFlKhrQQSjD0JGhL52LW9mlSDQv7Zc7hRf8BMhqtrjiWa5ITIRI8m6k
81ibplYSt3s87a9xFhX3PrXYkV7wCDHvRiFQNGhPit_vzA
S381ywqScfPHUwEkBGrfyCFNi4wHRLpJsJmXYDw-aEIBaXLa8h9YgaVz
LoT-IDIiUYGrlP0zh

In [4]:
# Limpiar estados
shutil.rmtree(execution_cache_dir)